In [7]:
import tensorflow as tf
import numpy as np

np.random.seed(444)

N = 10000
sigma = 0.1
noise = sigma * np.random.randn(N)
x = np.linspace(0, 2, N)
d = 3 + 2 * x + noise
d.shape = (N, 1)

# We need to prepend a column vector of 1s to `x`.
X = np.column_stack((np.ones(N, dtype=x.dtype), x))

mu = 0.01

def tf_descent(X_tf, d_tf, mu, N_epochs):
    N = X_tf.get_shape().as_list()[0]
    f = 2 / N

    w = tf.Variable(tf.zeros((2, 1)), name="w_tf")
    y = tf.matmul(X_tf, w, name="y_tf")
    e = y - d_tf
    grad = f * tf.matmul(tf.transpose(X_tf), e)

    training_op = tf.assign(w, w - mu * grad)
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        init.run()
        for epoch in range(N_epochs):
            sess.run(training_op)
        opt = w.eval()
    return opt

X_tf = tf.constant(X, dtype=tf.float32, name="X_tf")
d_tf = tf.constant(d, dtype=tf.float32, name="d_tf")

tf_w = tf_descent(X_tf, d_tf, mu, 10)
print(tf_w)

[[0.8283678]
 [0.9316438]]


In [20]:
ratings

[(0, 1),
 (1, 2),
 (2, 6),
 (3, 7),
 (4, 6),
 (5, 14),
 (6, 16),
 (7, 13),
 (8, 17),
 (9, 19),
 (10, 20),
 (11, 8),
 (12, 24),
 (13, 25),
 (14, 25),
 (15, 25),
 (16, 25),
 (17, 25),
 (18, 25),
 (19, 31),
 (20, 32),
 (21, 25),
 (22, 37),
 (23, 38),
 (24, 26),
 (25, 42),
 (26, 45),
 (27, 42),
 (28, 49),
 (29, 51),
 (30, 53),
 (31, 42),
 (32, 42),
 (33, 57),
 (34, 42),
 (35, 45),
 (36, 42),
 (37, 65),
 (38, 62),
 (39, 21),
 (40, 70),
 (41, 9),
 (42, 21),
 (43, 16),
 (44, 71),
 (45, 19),
 (46, 2),
 (47, 77),
 (48, 79),
 (49, 80),
 (50, 84),
 (51, 46),
 (52, 76),
 (53, 89),
 (54, 2),
 (55, 77),
 (56, 90),
 (57, 91),
 (58, 92),
 (59, 93),
 (60, 47),
 (61, 97),
 (62, 60),
 (63, 55),
 (64, 104),
 (65, 105),
 (66, 105),
 (67, 107),
 (68, 109),
 (69, 110),
 (70, 111),
 (71, 113),
 (72, 114),
 (73, 115),
 (74, 117),
 (75, 117),
 (76, 119),
 (77, 120),
 (78, 110),
 (79, 107),
 (80, 116),
 (81, 116),
 (82, 116),
 (83, 117),
 (84, 124),
 (85, 125),
 (86, 125),
 (87, 124),
 (88, 124),
 (89, 130),
 (9

In [ ]:
def choose_arm(self, t, unknown_article_ids, verbosity):
    """
    Choose an arm to pull = query to matched to infowavepage t that it has not been matched yet.
    :param t: page_id for queries to matched to.
    :param unknown_item_ids: Indexes of query ids that page t has not rated yet.
    :return: Received reward for matched query = 1/0 = page actually matched/unmatched query.
    """
    var_A = tf.get_variable("varA",[],dtype=tf.float64)
    var_b = tf.get_variable("varb",[],dtype=tf.float64)
    
    var_armfeatures = tf.get_variable("vararmf",[],dtype=tf.float64)
    arm_features = var_armfeatures[t]
    
    p_t = tf.Variable(tf.zeros((arm_features.shape[0], 1)), name="p_t")
    
    page_ids = unknown_article_ids

    
    for a in page_ids:  # iterate over all arms, which are pages

        x_ta = arm_features[a].reshape(arm_features[a].shape[0], 1)  # make a column vector
        A_a_inv = np.linalg.inv(A[a])
        theta_a = A_a_inv.dot(b[a])
        
        p_t[a] = theta_a.T.dot(x_ta) + self.alpha * np.sqrt(x_ta.T.dot(A_a_inv).dot(x_ta))
        
        
    if self.allow_selecting_known_arms:
        page_ids = range(self.num_articles)
        p_t += 9999

    for a in page_ids:  # iterate over all arms, which are pages
        x_ta = arm_features[a].reshape(arm_features[a].shape[0], 1)  # make a column vector
        
        #A_a_inv = np.linalg.inv(A[a])
        #theta_a = A_a_inv.dot(b[a])
        
        # solve the equation  theta_a = inv(A_a)*b_a
        
        theta_a = tf.squeeze(tf.linalg.solve(
              tf.eye(context_dim) + a_new, tf.expand_dims(b_new, axis=-1)),axis=-1)
        
        # p_t_a <- theta_a'*x_t_a +  alpha*sqrt(x_t_a'*inv(A_a)*x_t_a)
        
        # inv(A_a)*x_t_a
        g_t_a = tf.squeeze(tf.linalg.solve(tf.eye(context_dim)+a_new, tf.expand_dims(x_ta, axis=-1)), axis=-1)
        
        p_t[a] = tf.add(tf.tensordot(theta_a, x_ta)  + tf.tesnordot(alpha, tf.sqrt(tf.tensordot(x_ta, g_t_a))))

    max_p_t=tf.argmax(p_t, axis=0).eval()
     
    if max_p_t <= 0:
        print("Page {} has max p_t={}, p_t={}".format(t, max_p_t, p_t))

#     # I want to randomly break ties, np.argmax return the first occurence of maximum.
#     # So I will get all occurences of the max and randomly select between them
#     max_idxs = np.argwhere(p_t == max_p_t).flatten()
#     a_t = np.random.choice(max_idxs)  # idx of article to recommend to query t

    # observed reward = 1/0
    r_t = self.recommend(query_id=t, page_id=a_t, fixed_rewards=self.fixed_rewards, prob_reward_p=self.prob_reward_p)

    if verbosity >= 2:
        print("Query {} choosing item {} with p_t={} reward {}".format(t, a_t, p_t[a_t], r_t))

    x_t_at = arm_features[a_t].reshape(arm_features[a_t].shape[0], 1)  # make a column vector
    
    
    A[a_t] = A[a_t] + x_t_at.dot(x_t_at.T)
    b[a_t] = b[a_t] + r_t * x_t_at.flatten()  # turn it back into an array because b[a_t] is an array

    return r_t

In [1]:
import numpy as np
import pickle
import time
import tensorflow as tf
import math

#tf.executing_eagerly()
#tf.compat.v1.disable_eager_execution()

num_articles=321
num_arm_features=305

alpha=0.1


query_features=pickle.load(open('data/X_train_bow_features.pkl','rb'))
page_features=pickle.load(open('data/X_all_page_pca_features.pkl','rb'))
ratings = pickle.load(open('data/Y_train_labels.pkl','rb'))

ident = tf.eye(num_arm_features)
A = tf.Variable(tf.ones([num_articles, 1, 1], name='A') * ident)
b = tf.Variable(tf.zeros([num_articles, num_arm_features], name='b'))

# product = tf.matmul(ident3d[0,:], ident3d[0,:])

# #initialize the variable
#init_op = tf.initialize_all_variables()

def initialize_arm_features(queryf,pagef):
    X_query = tf.constant(queryf, dtype=tf.float32, name="X_query")
    X_page = tf.constant(pagef, dtype=tf.float32, name="X_page")
    return X_query, X_page

def get_arm_features(i,j):
    return tf.concat([X_query[i],X_page[j]],0)

def recommend(query_id, page_id, ratings, fixed_rewards=True, prob_reward_p=0.9, ):
    """
    Returns reward and updates rating maatrix self.R.
    :param fixed_rewards: Whether to always return 1/0 rewards for already rated items.
    :param prob_reward_p: Probability of returning the correct reward for already rated item.
    :return: Reward = either 0 or 1.
    """
    MIN_PROBABILITY = 0 # Minimal probability to like an item - adds stochasticity
    
#     print(query_id)
#     print(page_id)
    
    if ratings[query_id, page_id] == 1:
        if fixed_rewards:
            return 1
        else:
            return np.random.binomial(n=1, p=prob_reward_p)  # Bernoulli coin toss
    else:

        # the goal is to update a missing ""
        current_page_features = page_features[page_id,:] #get the article features
        current_query_features = query_features[query_id,:]  #get the article features

        # find out for a page, what query is rated as relevant (which for new query should be none)
        query_ratings = ratings[:,page_id]  #get all ratings by article id, it is a column
        query_pos_rat_idxs = np.argwhere(query_ratings == 1).flatten() # get all other positive ratings of the same article
        num_known_ratings = len(query_pos_rat_idxs)  # length of all other positive ratings

        match_likabilities=[]

        for query_idx in query_pos_rat_idxs:
            match_likabilities.append(cosine_similarity(current_query_features.reshape(-1,1), query_features[query_idx].reshape(-1,1)))

        result_match_likability = np.average(match_likabilities)

        if math.isnan(result_match_likability):
            result_match_likability=0

        binomial_reward_probability = result_match_likability
        #print (binomial_reward_probability)
        if binomial_reward_probability <= 0:
            #print("User={}, item={}, genre likability={}".format(user_id, item_id, result_genre_likability))
            binomial_reward_probability = MIN_PROBABILITY # this could be replaced by small probability

        approx_rating = np.random.binomial(n=1, p=binomial_reward_probability)  # Bernoulli coin toss

        if approx_rating == 1:
            ratings[query_id, page_id] = 1
        else:
            ratings[query_id, page_id] = 0

        #return approx_rating
        return approx_rating
    

#run the graph

t0 = time.time()
#with tf.compat.v1.Session() as sess:
X_query, X_page = initialize_arm_features(query_features, page_features)

#p_t = tf.Variable(tf.zeros([321,]), dtype=tf.float32, name="p_t")    
#tf.global_variables_initializer().run()
tf.compat.v1.global_variables_initializer()
#print the random values that we sample

for q_id in tf.range(600):
    axt = []
    for a in tf.range(321):
        x_ta = get_arm_features(q_id,a)
        A_new = A[a,:]
        b_new = b[a,:]
        theta_a = tf.squeeze(tf.linalg.solve(tf.eye(num_arm_features) + A_new, tf.expand_dims(b_new, axis=-1)),axis=-1)
        g_t_a = tf.squeeze(tf.linalg.solve(tf.eye(num_arm_features)+A_new, tf.expand_dims(x_ta, axis=-1)), axis=-1)
        #n_p = tf.add(tf.tensordot(theta_a, x_ta, (0,0)),tf.tensordot(alpha, tf.sqrt(tf.tensordot(x_ta, g_t_a, (0,0))),(0,0)))

        temp=tf.math.add(tf.tensordot(theta_a, x_ta,(0,0)),tf.sqrt(tf.tensordot(x_ta, g_t_a, (0,0))))
        axt.append(temp)
#         print(type(n_p))
#         print(sess.run(n_p))
       #(n_p).eval()
    p_t = tf.stack(axt)
    #print(sess.run(p_t))

    #print(p_t)
    max_p_t=tf.argmax(p_t, axis=0).numpy()
    #print(max_p_t)

    # need to add tile breaking
    a_t = max_p_t
    r_t = recommend(query_id=q_id, page_id=a_t, ratings=ratings)


    x_t_at = get_arm_features(0,a_t) 
    A_a_t = A[a_t,:]
    A_a_t_new = tf.add(A_a_t, tf.tensordot(x_t_at, x_t_at,(0,0)))
    b_a_t_new = tf.add(b[a_t],r_t*x_t_at)
#     print(sess.run(A_a_t_new))
#     print(A_a_t_new)
    tf.compat.v1.scatter_update(A,[a_t],A_a_t_new)
#     tf.compat.v1.scatter_nd_update()
#     A[a_t].assign(A_a_t_new).eval()
    tf.compat.v1.scatter_update(b,[a_t],b_a_t_new)
    #b[a_t] = b[a_t] + r_t * x_t_at.flatten()  # turn it back into an array because b[a_t] is an array
        
t1 = time.time()

# assign approach timing is 649 s
# append approach timing is 24 s


print("total timing is {}", str(t1-t0))

/home/syu/anaconda3/envs/tf2_gpu/lib/python3.7/site-packages/numpy/lib/function_base.py:390: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/syu/anaconda3/envs/tf2_gpu/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


total timing is {} 1573.923360824585


In [23]:
X_query.shape[0]
    

Dimension(600)

In [4]:
a = tf.constant([1,2.])
b = tf.constant([2,3.])
print(f"{tf.tensordot(a, b, 0)}\t tf.einsum('i,j', a, b)\t\t- ((the last 0 axes of a), (the first 0 axes of b))")
print(f"{tf.tensordot(a, b, ((),()))}\t tf.einsum('i,j', a, b)\t\t- ((() axis of a), (() axis of b))")
print(f"{tf.tensordot(b, a, 0)}\t tf.einsum('i,j->ji', a, b)\t- ((the last 0 axes of b), (the first 0 axes of a))")
print(f"{tf.tensordot(a, b, 1)}\t\t tf.einsum('i,i', a, b)\t\t- ((the last 1 axes of a), (the first 1 axes of b))")
print(f"{tf.tensordot(a, b, ((0,), (0,)))}\t\t tf.einsum('i,i', a, b)\t\t- ((0th axis of a), (0th axis of b))")
print(f"{tf.tensordot(a, b, (0,0))}\t\t tf.einsum('i,i', a, b)\t\t- ((0th axis of a), (0th axis of b))")



Tensor("Tensordot:0", shape=(2, 2), dtype=float32)	 tf.einsum('i,j', a, b)		- ((the last 0 axes of a), (the first 0 axes of b))
Tensor("Tensordot_1:0", shape=(2, 2), dtype=float32)	 tf.einsum('i,j', a, b)		- ((() axis of a), (() axis of b))
Tensor("Tensordot_2:0", shape=(2, 2), dtype=float32)	 tf.einsum('i,j->ji', a, b)	- ((the last 0 axes of b), (the first 0 axes of a))
Tensor("Tensordot_3:0", shape=(), dtype=float32)		 tf.einsum('i,i', a, b)		- ((the last 1 axes of a), (the first 1 axes of b))
Tensor("Tensordot_4:0", shape=(), dtype=float32)		 tf.einsum('i,i', a, b)		- ((0th axis of a), (0th axis of b))
Tensor("Tensordot_5:0", shape=(), dtype=float32)		 tf.einsum('i,i', a, b)		- ((0th axis of a), (0th axis of b))


In [17]:
v = tf.Variable(tf.constant([[1,1],[2,3]]))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    change_v = v[0,0].assign(4)
    print(sess.run(change_v))

[[4 1]
 [2 3]]


In [13]:
with tf.Session() as sess:
    a = tf.constant([1,2.])
    prod = tf.tensordot(a, a, 0)
    print(sess.run(prod))


[[1. 2.]
 [2. 4.]]


In [17]:
ident = tf.eye(num_arm_features)
ident2 = tf.ones([num_arm_features,num_arm_features])
A = tf.Variable(tf.ones([num_articles, 1, 1], name='A') * ident)

In [18]:
tf.compat.v1.scatter_nd_update(A,[0],ident2)

<tf.Tensor 'ScatterNdUpdate:0' shape=(321, 30, 30) dtype=float32_ref>

In [15]:
ident2

<tf.Tensor 'ones_1:0' shape=(30, 30) dtype=float32>